# PROVA INTELIGÊNCIA ARTIFICIAL

## 1 - (1 Ponto) Para cada um dos agentes a seguir, desenvolva uma descrição completando a tabela abaixo sobre o ambiente de tarefas de cada um.

(a) Agente comentarista de jogo.

(b) Agente sugestor de vinho.

(c) Agente treinador de time (como de basquete).


| Agente | Medida de desempenho                                 | Ambiente                             | Atuadores            | Sensores                             |
|--------|------------------------------------------------------|--------------------------------------|----------------------|--------------------------------------|
| (a)    | Tornar evidente as estratégias que estão acontecendo | Partida que está sendo realizada     | Emitir som de fala   | Imagens da partida                   |
| (b)    | Satisfação dos clientes do estabelecimento           | Estabelecimento onde se vende vinhos | Sugerir uma seleção de vinhos | Lista de vinhos disponiveis no local |
| (c)    | Vitória do time                                      | Partida do esporte em questão        | Instrução de táticas | Imagens da partida                   |

## 2 - (2 Pontos) Diferencie os 4 tipos de aprendizado de máquina. Para tanto, escolha uma base de dados do Kaggle e explique como aplicar os 4 métodos nessa mesma base.

A base escolhida foi '*spotify-music-analysis*' onde, através da API do Spotify, foram retiradas algumas informações sobre uma lista de músicas. Para mais informações sobre o significado dos atributos da base, entre em [AudioFeaturesObject](https://developer.spotify.com/documentation/web-api/reference/#/operations/get-several-audio-features).

### Aprendizagem Supervisionada
O objetivo do algoritmo de aprendizagem supervisionada é estabelecer um classificador que identifique as classes através de exemplos dados. Ou seja, dado um conjunto de exemplos que determina a classe rótulo, quando dado um novo conjunto de exemplos, o algoritmo deve ser capaz de classificar de acordo com as classes dos dados de treinamento.
Neste contexto da base 'spotify-music-analysis', pode se estabelecer como classe rótulo 'mode' onde se refere a dado aquele tom (*key*) possui um configuração de escala menor(0) ou maior(1).
Caso os atributos sejam contínuos, deve se usar o modelo preditivo regressão. Neste caso é mais apropriado o método preditivo classificação por conta dos atributos selecionados.

#### Exemplo de aplicação do modelo Classificação


In [82]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

data_frame = pd.read_csv("../db/data.csv")
data_frame = data_frame.drop("Unnamed: 0", axis="columns")

data_frame.drop(data_frame.columns.difference(['key','mode','valence']), 1, inplace=True)
data_frame.head(5)

,key,mode,valence
0,2,1,0.286
1,1,1,0.588
2,2,1,0.173
3,5,1,0.230
4,5,0,0.904


Plotaremos um diagrama pra podermos obter a classificação visualmente:

In [185]:
import plotly.express as px

df_sample = data_frame.sample(n=50)

mkey = []

for i in df_sample.index:
    if df_sample['mode'][i] == 1:
        mkey.append(df_sample['key'][i] + 0.5)
    else:
        mkey.append(df_sample['key'][i])


df_scatter = pd.DataFrame({
    'mkey' : mkey,
    'valence' : df_sample.get('valence')
})


fig  = px.scatter(df_sample, x = 'valence',y = 'mode', log_x = True, width = 800)
fig.update_traces(marker = dict(size = 12, line=dict(width = 2)), selector = dict(mode = 'markers'))
fig.update_layout(title = 'Diagram valence x mode')
fig.update_xaxes(title = 'Valence')
fig.update_yaxes(title = 'Mode')
fig.show()

O diagrama nos permite classificar essa amostragem, dada a escala, se aquela música vai ser mais alegre ou mais triste

### Aprendizagem Não Supervisionada

Neste tipo de aprendizagem, o indutor analisa os exemplos fornecidos e tentar classificar através de agrupamento.
Ainda na base *spotify-music-analysis*, um exemplo desse tipo de aprendizagem pode ser dado pelo atributo *danceability* onde descreve o qual recomendada é uma música para dançar. Isso é feito pelo agrupamento de elementos musicais como "*tempo, rhythm stability, beat strength, and overall regularity.*"